<a href="https://colab.research.google.com/github/zeeshan1047/research/blob/main/blue_score_of_final_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df =pd.read_excel('/content/drive/MyDrive/translation research/dataset/eu_dataset.xlsx')

# Inspect the dataframe
df.head(10)


,English,Urdu
0,is zain your nephew,زین تمہارا بھتیجا ہے۔
1,i wish youd trust me,کاش تم مجھ پر بھروسہ کرتے
2,did he touch you,کیا اس نے آپ کو چھوا؟
3,its part of life,اس کی زندگی کا حصہ
4,zain isnt ugly,زین بدصورت نہیں ہے۔
5,above all be patient,سب سے بڑھ کر صبر کرو
6,i learned it from him,میں نے اسے اس سے سیکھا۔
7,why am i doing this,میں یہ کیوں کر رہا ہوں
8,i made a bad decision,میں نے ایک برا فیصلہ کیا
9,zain wont care,زین پرواہ نہیں کرے گا


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Example dataset (replace with your data loading code)
english_text = df['English'].astype(str)
urdu_text = df['Urdu'].astype(str)

# Tokenization
english_tokenizer = Tokenizer( oov_token='<OOV>')
urdu_tokenizer = Tokenizer( oov_token='<OOV>')
english_tokenizer.fit_on_texts(english_text)
urdu_tokenizer.fit_on_texts(urdu_text)
#print(english_tokenizer.word_index)
#print(urdu_tokenizer.word_index)



In [ ]:
#import pickle
#import os


# Define the paths
#english_tokenizer_path = '/content/drive/MyDrive/translation research/english_to_urdu_rnn1_english_tokenizer1.pkl'
#urdu_tokenizer_path = '/content/drive/MyDrive/translation research/english_to_urdu_rnn1_urdu_tokenizer1.pkl'

# Save English tokenizer
#with open(english_tokenizer_path, 'wb') as file:
#    pickle.dump(english_tokenizer, file)

# Save Urdu tokenizer
#with open(urdu_tokenizer_path, 'wb') as file:
#    pickle.dump(urdu_tokenizer, file)

In [ ]:
input_vocab_size = len(english_tokenizer.word_index) + 1
target_vocab_size = len(urdu_tokenizer.word_index) + 1
#print(input_vocab_size)
#rint(target_vocab_size)


In [ ]:
# Sequencing

english_sequences = english_tokenizer.texts_to_sequences(english_text)
urdu_sequences = urdu_tokenizer.texts_to_sequences(urdu_text)

max_len = max(max(len(seq) for seq in english_sequences), max(len(seq) for seq in urdu_sequences))



In [ ]:
max_len = 234

In [ ]:
#padding
english_sequences_padded = pad_sequences(english_sequences, maxlen=max_len, padding='post')
urdu_sequences_padded = pad_sequences(urdu_sequences, maxlen=max_len, padding='post')



In [ ]:
#spliting traing and testing set
from sklearn.model_selection import train_test_split
train_english, val_english, train_urdu, val_urdu = train_test_split(
    english_sequences_padded, urdu_sequences_padded, test_size=0.2, random_state=42
)



In [ ]:
# Load the model with a smaller batch size
import tensorflow as tf
#with tf.device('/CPU:0'): # Loads the model on the CPU
best_model = tf.keras.models.load_model('/content/drive/MyDrive/translation research/bestrnn_model1.keras')

In [ ]:
history=best_model.history

In [ ]:
val_english_small = val_english[:800]  # Adjust the size as needed
val_urdu_small = val_urdu[:800]  # Adjust the size as needed
with tf.device('/CPU:0'):
  predictions = best_model.predict([val_english_small, val_urdu_small], batch_size=1)

800/800 [==============================] - 105s 131ms/step


In [ ]:
import re  # Import the 're' module for regular expression operations

predicted_sequences = predictions.argmax(axis=-1)
predicted_text = urdu_tokenizer.sequences_to_texts(predicted_sequences)
# Iterate through each predicted sentence and remove '<OOV>' if present
for i in range(len(predicted_text)):
    predicted_text[i] = re.sub(r'\s*<OOV>\s*', ' ', predicted_text[i]).strip()

formatted_output = "[\n"
for sentence in predicted_text:
    formatted_output += f"    '{sentence}',\n"
    predicted_output = formatted_output.rstrip(",\n") + "\n]"

# Print the formatted output
print(predicted_output)

[
    'کسی جان کو اس کی طاقت سے بڑھ کر تکلیف نہ دی جائے ، اور نہ ماں کو اس کے بچے کے باعث نقصان پہنچایا جائے اور نہ باپ کو اس کی اولاد کے سبب سے ،',
    'وہ خوبصورت ہے',
    'ٹھنڈا',
    'وہ میری بہن نہیں ہے۔',
    'زین ویسے بھی کہاں ہے؟',
    'بزدل نہ بنو',
    'اور اگر مسِیح نہیں جی اُٹھا تو تُمہارا اِیمان بےفائِدہ ہے تُم اَب تک اپنے گُناہوں میں گِرفتار ہو ۔',
    'گوگل کے ذریعے ترجمہ کم قیمتوں میں مزیدار کراہی۔',
    'بیشک جو لوگ اللہ اور اُس کے رسول صلی اللہ علیہ وآلہ وسلم سے عداوت رکھتے ہیں وہی ذلیل ترین لوگوں میں سے ہیں ۔',
    'زین گھبرا گیا',
    'آپ سخی ہیں',
    'جن لوگوں نے شعیب علیہ السلام کو جھٹلایا وہ ایسے نیست و نابود ہوئے گویا وہ اس بستی میں کبھی بسے ہی نہ تھے ۔ جن لوگوں نے شعیب علیہ السلام کو جھٹلایا حقیقت میں وہی نقصان اٹھانے والے ہوگئے ۔',
    'مجھے موسیقی سننا پسند ہے',
    'گوگل کے ذریعہ ترجمہ شدہ غسل کا بٹن',
    'میرے ماموں نے مجھے یہ گھڑی دی تھی۔',
    'اس کے پاس پچاس ڈالر تھے۔',
    'اس نے اس کو اپنے سارے پیسے دے دیئے۔',
    'اِس کے ساتھ ساتھ کل گیندیں بھی چوری

In [ ]:
val_urdu_small
import re  # Import the 're' module for regular expression operations

text = urdu_tokenizer.sequences_to_texts(val_urdu_small)
for i in range(len(text)):
    text[i] = re.sub(r'\s*<OOV>\s*', ' ', text[i]).strip()

# Format the sentences in the desired format
formatted_output = "[\n"
for sentence in text:
    formatted_output += f"    ['{sentence}'],\n"
    validation_output = formatted_output.rstrip(",\n") + "\n]"

# Print the formatted output
print(validation_output)

[
    ['کسی جان کو اس کی طاقت سے بڑھ کر تکلیف نہ دی جائے ، اور نہ ماں کو اس کے بچے کے باعث نقصان پہنچایا جائے اور نہ باپ کو اس کی اولاد کے سبب سے ،'],
    ['وہ خوبصورت ہے'],
    ['ٹھنڈا'],
    ['وہ میری بہن نہیں ہے۔'],
    ['زین ویسے بھی کہاں ہے؟'],
    ['بزدل نہ بنو'],
    ['اور اگر مسِیح نہیں جی اُٹھا تو تُمہارا اِیمان بےفائِدہ ہے تُم اَب تک اپنے گُناہوں میں گِرفتار ہو ۔'],
    ['گوگل کے ذریعے ترجمہ کم قیمتوں میں مزیدار کراہی۔'],
    ['بیشک جو لوگ اللہ اور اُس کے رسول صلی اللہ علیہ وآلہ وسلم سے عداوت رکھتے ہیں وہی ذلیل ترین لوگوں میں سے ہیں ۔'],
    ['زین گھبرا گیا'],
    ['آپ سخی ہیں'],
    ['جن لوگوں نے شعیب علیہ السلام کو جھٹلایا وہ ایسے نیست و نابود ہوئے گویا وہ اس بستی میں کبھی بسے ہی نہ تھے ۔ جن لوگوں نے شعیب علیہ السلام کو جھٹلایا حقیقت میں وہی نقصان اٹھانے والے ہوگئے ۔'],
    ['مجھے موسیقی سننا پسند ہے'],
    ['گوگل کے ذریعہ ترجمہ شدہ غسل کا بٹن'],
    ['میرے ماموں نے مجھے یہ گھڑی دی تھی۔'],
    ['اس کے پاس پچاس ڈالر تھے۔'],
    ['اس نے اس کو اپنے سارے پیسے دے دیئے۔'],
    ['

In [ ]:
import pickle

# Define the paths
english_tokenizer_path = '/content/drive/MyDrive/translation research/english_to_urdu_rnn1_english_tokenizer1.pkl'
urdu_tokenizer_path = '/content/drive/MyDrive/translation research/english_to_urdu_rnn1_urdu_tokenizer1.pkl'

# Load English tokenizer
with open(english_tokenizer_path, 'rb') as file:
    english_tokenizer = pickle.load(file)

# Load Urdu tokenizer
with open(urdu_tokenizer_path, 'rb') as file:
    urdu_tokenizer = pickle.load(file)

In [ ]:
import nltk
from nltk.translate.bleu_score import corpus_bleu
import numpy as np

# Ensure NLTK data is downloaded
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:



def compute_bleu_score(predictions, references):
    """Compute BLEU score using nltk."""
    # Tokenize the predictions and references using nltk.word_tokenize
    tokenized_predictions = [nltk.word_tokenize(prediction) for prediction in predictions]

    # Ensure references is a list of lists of tokens
    tokenized_references = [[nltk.word_tokenize(ref[0])] for ref in references]  # Access the sentence within the inner list

    # Compute BLEU score
    bleu_score = corpus_bleu(tokenized_references, tokenized_predictions)
    return bleu_score

# Example predictions and references
# Convert predicted_output and validation_output to lists of sentences
predictions = eval(predicted_output)  # Evaluate the string representation of the list
references = eval(validation_output)  # Evaluate the string representation of the list

# Compute BLEU score
bleu_score = compute_bleu_score(predictions, references)
print(f"BLEU score: {bleu_score:.4f}")

BLEU score: 0.9274


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from collections import Counter
from sklearn.metrics import confusion_matrix



In [ ]:
def tokenize(texts):
    """Tokenize texts into lists of tokens."""
    return [nltk.word_tokenize(text) for text in texts]

def flatten(tokens_list):
    """Flatten a list of tokenized texts."""
    return [token for sublist in tokens_list for token in sublist]

def compute_confusion_matrix(predictions, references):
    """Compute confusion matrix for token-level predictions and references."""
    # Tokenize the predictions and references
    tokenized_predictions = tokenize(predictions)
    tokenized_references = tokenize(references)

    # Flatten tokenized lists
    flat_predictions = flatten(tokenized_predictions)
    flat_references = flatten(tokenized_references)

    # Create confusion matrix
    # We use unique tokens as classes
    labels = sorted(set(flat_predictions + flat_references))
    cm = confusion_matrix(flat_references, flat_predictions, labels=labels)

    return cm, labels

def tokenize(texts):
    """Tokenize texts into lists of tokens."""
    return [nltk.word_tokenize(text[0]) if isinstance(text, list) else nltk.word_tokenize(text) for text in texts]

def compute_precision_recall_f1(predictions, references):
    """Compute precision, recall, and F1 score."""
    # Tokenize the predictions and references
    tokenized_predictions = tokenize(predictions)
    tokenized_references = tokenize(references)

    # Flatten tokenized lists
    flat_predictions = [token for sublist in tokenized_predictions for token in sublist]
    flat_references = [token for sublist in tokenized_references for token in sublist]

    # Create a set of all tokens to use for precision, recall, and F1 score calculation
    all_tokens = set(flat_predictions + flat_references)

    # Create reference and prediction counts
    ref_counts = Counter(flat_references)
    pred_counts = Counter(flat_predictions)

    # Calculate precision, recall, and F1 score
    precision = []
    recall = []
    f1 = []

    for token in all_tokens:
        ref_count = ref_counts.get(token, 0)
        pred_count = pred_counts.get(token, 0)
        true_positives = min(ref_count, pred_count)

        # Precision = TP / (TP + FP)
        prec = true_positives / (pred_count + 1e-10)  # Add a small number to avoid
         # Recall = TP / (TP + FN)
        rec = true_positives / (ref_count + 1e-10)
        # F1 = 2 * (Precision * Recall) / (Precision + Recall)
        f1_score = 2 * (prec * rec) / (prec + rec + 1e-10)

        precision.append(prec)
        recall.append(rec)
        f1.append(f1_score)

    # Average the metrics
    avg_precision = sum(precision) / len(precision)
    avg_recall = sum(recall) / len(recall)
    avg_f1 = sum(f1) / len(f1)

    return avg_precision, avg_recall, avg_f1

# Compute Precision, Recall, and F1 Score
precision, recall, f1 = compute_precision_recall_f1(predictions, references)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
# Compute confusion matrix
cm, labels = compute_confusion_matrix(predictions, references)

print("Confusion Matrix:")
print(cm)
#print("Labels:")
#print(labels)

Precision: 0.7797
Recall: 0.7801
F1 Score: 0.7797
Confusion Matrix:
[[  1   0   0 ...   0   0   0]
 [  0   1   0 ...   0   0   0]
 [  0   0   1 ...   0   0   0]
 ...
 [  0   0   0 ...   9   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0 317]]


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
test_english_small = val_english[100:115]  # Adjust the size as needed
test_urdu_small = val_urdu[100:115]  # Adjust the size as needed

prediction = best_model.predict([test_english_small, test_urdu_small], batch_size=1)

# Postprocess
import re  # Import the 're' module for regular expression operations

predicted_sequences = prediction.argmax(axis=-1)
predicted_text = urdu_tokenizer.sequences_to_texts(predicted_sequences)
# Iterate through each predicted sentence and remove '<OOV>' if present
for i in range(len(predicted_text)):
    predicted_text[i] = re.sub(r'\s*<OOV>\s*', ' ', predicted_text[i]).strip()

formatted_output = "[\n"
for sentence in predicted_text:
    formatted_output += f"    '{sentence}',\n"
    predicted_output = formatted_output.rstrip(",\n") + "\n]"

# Print the formatted output
print(predicted_output)



15/15 [==============================] - 1s 19ms/step
[
    'میں وہ کروں گا',
    'ایک جھوٹ دوسرے کی طرف لے جاتا ہے۔',
    'وہ اُسے بلا رہی ہے جس نے حق سے پیٹھ پھیرے اور رُوگردانی کی ۔',
    'پیارے میں تم سے پیار کرتا ہوں۔',
    'گھر پھنس گیا',
    'میں بہت قدامت پسند ہوں۔',
    'کیونکہ خُدا کا وہ فضل ظاہِر ہُؤا ہے جو سب آدمِیوں کی نِجات کا باعِث ہے ۔',
    'کیا یہ فلیٹ نہیں ہے؟',
    'اور ہم نے ان کے پیچھے اس دنیا میں بھی لعنت لگا دی اور قیامت کے دن بھی وہ بجھنے لوگوں میں شمار ہوں گے ۔',
    'میرے پاس زینز کی چابیاں ہیں۔',
    'ایک سیکنڈ پر پکڑیں',
    'گوگل کے ذریعہ ترجمہ کیا گیا یا انتہائے نہیں مچھلی ہے تو فطری بل بنے یہ',
    'اے لوگو زمین کی چیزوں میں سے جو حلال اور پاکیزہ ہے کھاؤ ، اور شیطان کے راستوں پر نہ چلو ، بیشک وہ تمہارا کھلا دشمن ہے ۔',
    'اور جِس کِسی نے گھروں یا بھائیوں یا بہنوں یا باپ یا ماں یا بچّوں یا حُکومتیں کو میرے نام کی خاطِر چھوڑ دِیا ہے اُس کو بِچھُّو مِلے گا اور ہمیشہ کی زِندگی کا وارِث ہوگا ۔',
    'میں ابھی آرہا ہوں۔'
]


In [ ]:

text = urdu_tokenizer.sequences_to_texts(test_urdu_small)
# Iterate through each predicted sentence and remove '<OOV>' if present
for i in range(len(predicted_text)):
    text[i] = re.sub(r'\s*<OOV>\s*', ' ',text[i]).strip()

formatted_output = "[\n"
for sentence in text:
    formatted_output += f"    '{sentence}',\n"
    text = formatted_output.rstrip(",\n") + "\n]"

# Print the formatted output
print(text)

[
    'میں وہ کروں گا',
    'ایک جھوٹ دوسرے کی طرف لے جاتا ہے۔',
    'وہ اُسے بلا رہی ہے جس نے حق سے پیٹھ پھیری اور رُوگردانی کی ۔',
    'پیارے میں تم سے پیار کرتا ہوں۔',
    'گھر پھنس گیا',
    'میں بہت قدامت پسند ہوں۔',
    'کیونکہ خُدا کا وہ فضل ظاہِر ہُؤا ہے جو سب آدمِیوں کی نِجات کا باعِث ہے ۔',
    'کیا یہ فلیٹ نہیں ہے؟',
    'اور ہم نے ان کے پیچھے اس دنیا میں بھی لعنت لگا دی اور قیامت کے دن بھی وہ بدحال لوگوں میں شمار ہوں گے ۔',
    'میرے پاس زینز کی چابیاں ہیں۔',
    'ایک سیکنڈ پر پکڑیں',
    'گوگل کے ذریعہ ترجمہ کیا گیا یا متلب نہیں مچھلی ہے تو نازائز بل بنے یہ',
    'اے لوگو زمین کی چیزوں میں سے جو حلال اور پاکیزہ ہے کھاؤ ، اور شیطان کے راستوں پر نہ چلو ، بیشک وہ تمہارا کھلا دشمن ہے ۔',
    'اور جِس کِسی نے گھروں یا بھائیوں یا بہنوں یا باپ یا ماں یا بچّوں یا کھیتّوں کو میرے نام کی خاطِر چھوڑ دِیا ہے اُس کو سوگُنا مِلے گا اور ہمیشہ کی زِندگی کا وارِث ہوگا ۔',
    'میں ابھی آرہا ہوں۔'
]


In [ ]:
best_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 decoder_input (InputLayer)  [(None, 234)]                0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, 234, 512)             1027174   ['decoder_input[0][0]']       
                                                          4                                       
                                                                                                  
 simple_rnn_1 (SimpleRNN)    (None, 234, 512)             524800    ['embedding_1[0][0]']         
                                                                                                  
 encoder_input (InputLayer)  [(None, 234)]                0         []                        